In [1]:
import pandas as pd
import wrangle
import new_wrangle
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler

# acquire data

Basic acquire data, we are not modifying anything. We just wan to see the data .

In [ ]:
# use function to acquire data
df1= wrangle.acquire_data()

In [ ]:
#check  info
df1.info()

In [ ]:
df1.shape

In [ ]:
df1= new_wrangle.change_columns(df1)

In [ ]:
df1 = new_wrangle.this_week_next_week_lagger(df1)

In [ ]:
df1 = new_wrangle.add_which_holiday(df1)

In [ ]:
df1 =new_wrangle.add_which_holiday(df1)

In [ ]:
df1 = new_wrangle.create_dummies(df1)

In [ ]:
df1 = new_wrangle.get_new_index(df1)

In [ ]:
df1.info()

In [ ]:
def rolling_delta(df):
    
    '''
    This function will calculate rolling monthly averages for CPI, f
    fuel_price & Unemployment and reference these rolling avgs on weekly periods to calculate a MoM % change.
    NOTE: Due to our modified index, the initial calcs for each store reference prvs store end data due to sequencing.
    This issue is resolved 8 periods into the time series for each store when enough data for new store is in place. 
    '''
    
    # calc monthly rolling avgs for each feature
    df['fuel_4wk_rolling'] = df.fuel_price.rolling(4).mean()
    df['cpi_4wk_rolling'] = df.CPI.rolling(4).mean()
    df['unemp_4wk_rolling'] = df.this_week_unemployment.rolling(4).mean()
    
    # calc percent month-on-month delta on weekly monthly avg observation
    df['avgMoM_perc_fuel'] = df.fuel_4wk_rolling.pct_change(4)
    df['avgMoM_perc_cpi'] = df.cpi_4wk_rolling.pct_change(4)
    df['avgMoM_perc_unemp'] = df.unemp_4wk_rolling.pct_change(4)
    
    # calc quarterly rolling avgs for each feature
    df['fuel_quarterly_rolling'] = df.fuel_price.rolling(12).mean()
    df['cpi_quarterly_rolling'] = df.CPI.rolling(12).mean()
    df['unemp_quarterly_rolling'] = df.this_week_unemployment.rolling(12).mean()
    
    # calc percent quarter-on-quarter delta on weekly monthly avg observation
    df['avgQoQ_perc_fuel'] = df.fuel_quarterly_rolling.pct_change(12)
    df['avgQoQ_perc_cpi'] = df.cpi_quarterly_rolling.pct_change(12)
    df['avgQoQ_perc_unemp'] = df.unemp_quarterly_rolling.pct_change(12)
    
    return df

In [ ]:
df5 = rolling_delta(df1)

In [ ]:
df5.info()

In [ ]:
df5.isnull().sum()

In [ ]:
train, test,  X_train_scaled, X_test_scaled, y_train, y_test =new_wrangle.split_scale(df5, 'next_week_sales_target', MinMaxScaler())

In [ ]:
X_train_scaled.isnull().sum()

In [ ]:
def get_season(now_date):
    '''
    This function gets the season from a dateteime
    '''
    
    Y = 2000 # dummy leap year to allow input X-02-29 (leap day)
    seasons = [('winter', '2000-01-01',  '2000-03-01'),
           ('spring', '2000-03-21',  '2000-06-20'),
           ('summer', '2000-06-21',  '2000-09-22'),
           ('fall', '2000-09-23',  '2000-12-20'),
           ('winter', '2000-12-21',  '2000-12-31')]
        
    now = now_date.replace(year, Y)
    
    season = next(season for season, (start, end) in seasons if start <= now <= end)
    
    return season


def season_column(df):
    '''
    This function creates two new columns 
    Season for this week date 
    And season for next week date
    Uses get_season function
    '''

    df['next_week_season'] = df.dropna().next_week_date.apply(get_season)

    df['this_week_season'] = df.dropna().this_week_date.apply(get_season)

    return df

In [ ]:
df1.info()

In [ ]:
type(df.this_week_date)

In [ ]:
l = [1,2,3]

In [ ]:
pd.Series(l)

In [ ]:
p = ['next_week_date'].astype('string')[0][5:]

In [ ]:
# l=[]
# for r in range (len (df1)):
#    fp= df1['next_week_date'].astype('string')[r][5:]
#     da= '2020'+ fp
    l= l+ da

In [ ]:
l

In [ ]:
get_season(df1)

**Takeaways**
- we have 10 columns and 6435 columns
- no nulls
- we nned to change some columns type

# Ouliers store_size

We are exploring the store_type vs store_size, we notice that there are outliers

In [ ]:
sns.boxplot(x= df1['Type'], y= df1['Size'])

In [ ]:
#check only the outliers
sns.boxplot(x= df1['Type'], y= df1['Size'])
plt.ylim(0,46000)

we can see there are outliers for B and A and we decided to change those to  store type C

In [ ]:
#getting a df that has all store_size < 50000 and df.store_type != "C"
df3 = df1 [(df1.Size < 50000) & (df1.Type != "C")]

In [ ]:
#lets see which stores type A  are df.store_size < 50000
df3[df3.Type == "A"].groupby('Store').count()

In [ ]:
#lets see which stores type A  are df.store_size < 50000
df3[df3.Type == "B"].groupby('Store').count()

**Takeaways**
- Instead of removing theses outliers, we decided to change them to the type based on the size.
- I gave the store_id to Natasha so she is going to change the type in the wrangle function.


# Wrangle

the wrangle_walmar function is renaming columns, adding new columns, dummi variables, and set date as index (datetime type)

In [ ]:
#using the funcion wrangle
df= new_wrangle.wrangle_walmart()

In [ ]:
#chec the data
df.info()

In [ ]:
df.head(1)

In [ ]:
#checking if the store_type has no outliers
sns.boxplot(x= df['store_type'], y= df['store_size'])

In [ ]:
df.select_dtypes(exclude = ['datetime64[ns]', 'object'])

In [ ]:
df.info()

In [ ]:
def distribution (df):
    '''
    takes in a df and plot individual variable distributions excluding object type
    '''
    plt.figure
    plt.style.use("ggplot")
    df2 = df.select_dtypes(exclude = ['datetime64[ns]', 'object'])
    cols =df2.columns.to_list()
    for col in cols:

        plt.hist(df[col],color ='blue')
        plt.title(f'Distribution of {col}')
        plt.xlabel(col)
        plt.ylabel('Number of Weeks ')
        plt.xticks(rotation=45)
        plt.ticklabel_format(style = 'plain')

        plt.show()

In [ ]:
distribution(df)

In [ ]:
df.groupby('store_type').mean()

In [ ]:
#let see distribution by store type
store_type = ["A", "B", "C"]
for store in store_type:
    print("Store type: ", store)
    distribution(df[df.store_type == store])
    

In [ ]:
#how many stores are Type C
len(df[df.store_type == 'C'].store_id.value_counts())

In [ ]:
# storeid for type c
df[df.store_type == 'C'].store_size.value_counts()

In [ ]:
#store if for type B
df[df.store_type == 'B'].store_size.value_counts().sort_index()

In [ ]:
#how many stores are Type B
len(df[df.store_type == 'B'].store_id.value_counts().sort_index())

In [ ]:
#store_Id for type A
df[df.store_type == 'A'].store_size.value_counts().sort_index()

In [ ]:
##how many stores are Type A
len(df[df.store_type == 'A'].store_id.value_counts().sort_index())

In [ ]:
df.info()

In [ ]:
df.shape

**Takeaways**
- after wrangle_walmar we ended with 17 columns, and 6435 observations ( we have nulls)

<hr style="border:2px solid blue"> </hr>

## the following steps were done to complete the wrangle function

# dummies 

we are going to create dummies for :
- Holiday_name


In [ ]:
#create dummy variables for 5 columns
def create_multiple_dummies (df, dumm_col = ['holiday_name', 'season', 'store_type', 'month', 'year']):
    '''
    Takes in a df and columns to create dummies.
    retunr the original df with de new columns (dimmies)
    '''
    #the column year is an integer we need to conver as string
    df['year']= df['year'].astype('string')
    #create dummy variables 
    for col in dumm_col:
        #create dummies
        df_dummies = pd.get_dummies(df[col], dummy_na=False)
        #  concat df_dummies with my df
        df = pd.concat([df, df_dummies], axis =1)
    #drop no_holiday columns and year
    df = df.drop(columns = ['no_holiday', 'year'])
    return df

In [ ]:
def create_dummies (df, dumm_col = ['holiday_name']):
    '''
    Takes in a df and columns to create dummies.
    retunr the original df with de new columns (dummies)
    '''
    #create dummy variables 
    for col in dumm_col:
        #create dummies
        df_dummies = pd.get_dummies(df[col], dummy_na=False)
        #  concat df_dummies with my df
        df = pd.concat([df, df_dummies], axis =1)
    #drop no_holiday columns and year
    df = df.drop(columns = ['no_holiday'])
    return df

In [ ]:
df['holiday_name'].value_counts()

In [ ]:
#test the function before I add it to .py file
df2 = create_dummies(df)

In [ ]:
df2.info()

In [ ]:
df2.columns

# Create new columns

#we need to create new columns  pre- christmas  and tax_season because on exploration(Alberto and Heather ) they found a peak in those dates. 

In [ ]:
#checking the previous weeks for christmas

In [ ]:
df2.head(1)

create a new column for is pre_christmas

-Christmas:

    - 31-Dec-10,  (pre_christmas = 24-Dec-10, 17-Dec 10)
    - 30-Dec-11, (pre_christmas = 23-Dec-11, 16-Dec 11)

In [ ]:
# create new colum and add zeros to everything
df2 ['pre_christmas'] = 0

In [ ]:
df2.columns

In [ ]:
#getting the list for pre_christmas
pre_c= ['2010-12-24', '2010-12-17', '2011-12-23', '2011-12-16']

In [ ]:
#add value 1 for only pre_christmas weeks
#df2.loc[pre_c, 'pre_christmas'] = 1

In [ ]:
#CHECK THE VALUES
#df2['pre_christmas'].value_counts()

#### ADD TAX SEASON

###  first 2 weeks of April
- 2010-04-02 & 2010-04-09
- 2011-04-01 & 2011-04-08
- 2012-04-06 & 2012-04-13

In [ ]:
#crete a new column and assign 0 as value
df2['tax_season'] = 0 

In [ ]:
#getting the list for tax
tax= ['2010-04-02 ', '2010-04-09', '2011-04-01', '2011-04-08', '2012-04-06', '2012-04-13']

In [ ]:
#add value 1 for only for the list above
df2.loc[tax, 'tax_season'] = 1

In [ ]:
#CHECK THE VALUES
df2['tax_season'].value_counts()

# Scale

In [ ]:
df2.head(1)

**note :** before scaling we need to split our data


In [ ]:
#select only the numeric columns 
num_df = df2.select_dtypes(exclude='object')

In [ ]:
num_df.shape

In [ ]:
num_df.head()

In [ ]:
#split using a function
train, test, X_train, y_train, X_test, y_test = new_wrangle.train_test(num_df, 'weekly_sales')

In [ ]:
# imports to scale
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler

In [ ]:
def scaled_df ( train_df , test_df, columns,  scaler):
    '''
    Take in a 3 df and a type of scaler that you  want to  use. it will scale all columns
    except object type. Fit a scaler only in train and tramnsform in train, validate and test.
    returns  new dfs with the scaled columns.
    scaler : MinMaxScaler() or RobustScaler(), StandardScaler() 
    Example:
    scaled_df( X_train , X_test, columns , RobustScaler())
    
    '''
    #import
    from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler
    # fit our scaler
    scaler.fit(train_df[columns])
    # get our scaled arrays
    train_scaled = scaler.transform(train_df[columns])
    test_scaled= scaler.transform(test_df[columns])

    # convert arrays to dataframes
    train_scaled_df = pd.DataFrame(train_scaled, columns=columns).set_index([train_df.index.values])
    test_scaled_df = pd.DataFrame(test_scaled, columns=columns).set_index([test_df.index.values])

#     #add the columns that are not scaled
#     train_scaled_df = pd.concat([train_scaled_df, train_df.drop(columns = columns) ], axis= 1 )
#     test_scaled_df = pd.concat([test_scaled_df, test_df.drop(columns = columns) ], axis= 1 )
    #plot
    for col in columns: 
        plt.figure(figsize=(13, 6))
        plt.subplot(121)
        plt.hist(train_df[col], ec='black')
        plt.title('Original')
        plt.xlabel(col)
        plt.ylabel("counts")
        plt.subplot(122)
        plt.hist(train_scaled_df[col],  ec='black')
        plt.title('Scaled')
        plt.xlabel(col)
        plt.ylabel("counts")



    return train_scaled_df,  test_scaled_df

In [ ]:
columns =  X_train.columns.to_list()

In [ ]:
#test the function
#train_Xscaled_df,  test_Xscaled_df= scaled_df( X_train , X_test, columns , MinMaxScaler())

In [ ]:
def train_test(df, target):
    '''
    This function brings in the dataframe and the target feature
    then returns X_train, y_train, X_test and y_test with their respective shapes
    '''
    train = df[:'05-2012'] # includes everything until june 2016
    test = df['06-2012':"2012"] #includes last 6 months

    # split train into X (dataframe, drop target) & y (series, keep target only)
    X_train = train.drop(columns=[target])
    y_train = train[target]

    # split test into X (dataframe, drop target) & y (series, keep target only)
    X_test = test.drop(columns=[target])
    y_test = test[target]

    # Have function print datasets shape
    print(f'X_train -> {X_train.shape}')
    print(f'X_test -> {X_test.shape}')

    return train, X_train, y_train, X_test, y_test

In [ ]:
def split_scale and scale (df, target, scaler):
    '''
    takes in a df and creates dummy variables, select only the numeric columns and  split into X_train, y_train, 
    X_test, y_test and scaled X_train, X_test.
    return   X_train_scaled, y_train_scaled, X_test, y_test
    '''



    #split
    train, X_train, y_train, X_test, y_test = train_test(num_df, target)

    #select the columns to scale
    columns =  X_train.select_dtypes(exclude='object').columns.to_list()
    #scale 
    X_train_scaled, X_test_scaled = scaled_df( X_train , X_test, columns , scaler)

    return train, X_train_scaled, X_test_scaled, y_train, y_test
        
        
        
        
    

In [ ]:
#split
train, test, X_train, y_train, X_test, y_test = new_wrangle.train_test(df, 'weekly_sales')

In [ ]:
#select the columns to scale
col =  X_train.select_dtypes(exclude='object').columns.to_list()

In [ ]:
col

In [ ]:
#scale 
X_train_scaled, X_test_scaled = scaled_df( X_train , X_test, col , MinMaxScaler())

In [ ]:
X_train_scaled.info()

In [ ]:
train, X_train_scaled, y_train_scaled, X_test, y_test = prepare_modeling_dummies (df, 'weekly_sales', MinMaxScaler())

In [ ]:
train.shape

In [ ]:
X_train_scaled.shape

# split 

In [ ]:
train, test,  X_train_scaled, X_test_scaled, y_train, y_test = new_wrangle.split_scale(df, 'weekly_sales', MinMaxScaler())

In [ ]:
X_train_scaled.info()

<hr style="border:2px solid blue"> </hr>

# Wrangle and split

let's test the functions wrangle and split and scale

In [2]:
df =new_wrangle.wrangle_walmart()

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6435 entries, 2010-02-05_store_1_2010-02-12 to 2012-10-26_store_45_nan
Data columns (total 34 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   store_id                 6435 non-null   object        
 1   this_week_date           6435 non-null   datetime64[ns]
 2   this_week_sales          6435 non-null   float64       
 3   this_week_holiday_flag   6435 non-null   int64         
 4   temperature              6435 non-null   int64         
 5   fuel_price               6435 non-null   float64       
 6   CPI                      6435 non-null   float64       
 7   this_week_unemployment   6435 non-null   float64       
 8   store_type               6435 non-null   object        
 9   store_size               6435 non-null   int64         
 10  next_week_1_year_ago     4140 non-null   float64       
 11  next_week_sales_target   6390 non-null   float64     

**Note:**  we have nulls in our df because we created a new column next_week_1_year_ago  that's why the fisrt year has nulls because we dond have information about the previous year

In [4]:
df.shape

(6435, 34)

In [5]:
df.next_week_holiday_name.value_counts()

no_holiday       5805
pre_christmas     180
super_bowl        135
labor_day         135
thanksgiving       90
christmas          90
Name: next_week_holiday_name, dtype: int64

In [6]:
train, test,  X_train_scaled, X_test_scaled, y_train, y_test = new_wrangle.split_scale(df, 'next_week_sales_target', MinMaxScaler())

train -> (2866, 32)
test -> (1229, 32)


In [7]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2866 entries, 2011-09-16_store_12_2011-09-23 to 2011-09-16_store_40_2011-09-23
Data columns (total 32 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   store_id                 2866 non-null   object 
 1   this_week_sales          2866 non-null   float64
 2   this_week_holiday_flag   2866 non-null   int64  
 3   temperature              2866 non-null   int64  
 4   fuel_price               2866 non-null   float64
 5   CPI                      2866 non-null   float64
 6   this_week_unemployment   2866 non-null   float64
 7   store_type               2866 non-null   object 
 8   store_size               2866 non-null   int64  
 9   next_week_1_year_ago     2866 non-null   float64
 10  next_week_sales_target   2866 non-null   float64
 11  next_week_holiday_flag   2866 non-null   float64
 12  next_week_season         2866 non-null   object 
 13  this_week_season         286

In [8]:
X_train_scaled.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2866 entries, 2011-09-16_store_12_2011-09-23 to 2011-09-16_store_40_2011-09-23
Data columns (total 26 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   this_week_sales          2866 non-null   float64
 1   this_week_holiday_flag   2866 non-null   float64
 2   temperature              2866 non-null   float64
 3   fuel_price               2866 non-null   float64
 4   CPI                      2866 non-null   float64
 5   this_week_unemployment   2866 non-null   float64
 6   store_size               2866 non-null   float64
 7   next_week_1_year_ago     2866 non-null   float64
 8   next_week_holiday_flag   2866 non-null   float64
 9   christmas                2866 non-null   float64
 10  labor_day                2866 non-null   float64
 11  pre_christmas            2866 non-null   float64
 12  super_bowl               2866 non-null   float64
 13  thanksgiving             286

In [ ]:
y_train.isnull().sum()

# Explore 

In [ ]:
df.shape

In [ ]:
train.shape

In [ ]:
train.head(1)

In [ ]:
train.info()

In [ ]:
#explore our target
plt.figure(figsize=(8, 10))
sns.histplot(train,
                 x=train['next_week_sales_target'],
                 #hue='loan_status',
                 multiple='layer'
                 )
plt.title("Distribution of Next week sales")
plt.xlabel('Weekly sales in dollars')
plt.ylabel('Number of weeks')
plt.xticks(rotation=45)
plt.ticklabel_format(style = 'plain')

plt.show()

In [ ]:
plt.figure(figsize=(10,6))
sns.boxplot(train['next_week_sales_target'])
plt.xticks(rotation=45)
#plt.ticklabel_format(style = 'plain')
plt.show()

In [ ]:
plt.figure(figsize=(10,6))
sns.boxplot(train['next_week_sales_target'])
plt.xlim(2500000)
plt.show()

In [ ]:
plt.figure(figsize=(15, 15))
sns.heatmap(train.corr(), cmap='coolwarm', annot=True, linewidth=0.5)
plt.show()

In [ ]:
#check the relation of my target with all the featues

In [ ]:
#let's focus on my target
plt.figure(figsize=(8, 12))
heatmap = sns.heatmap(train.corr()[['next_week_sales_target']].sort_values(by='next_week_sales_target', ascending=False), vmin=-3, vmax=3, annot=True, cmap='coolwarm')

**takeaways**
- next_week_1_year_ago, this_week_sales and store_size have stroger correlation with out target
- pre_christmas has also a positive correlation
- 
- negative correlation this_week_unemployment 

## Exploration target vs Fearture

### -  weekly_sales vs store_size

In [ ]:
plt.figure(figsize=(16, 10))
sns.jointplot(x= 'weekly_sales', y= 'store_size', data = train, hue ='season')

**takeaways**
- the small the store, less weekly sales
- winter has the greater weekly sales

### -  weekly_sales vs temperature

In [ ]:
plt.figure(figsize=(12, 10))
sns.jointplot(x= 'weekly_sales', y= 'temperature', data = train, hue= "season")
plt.show()

**Takeaways**
- Winter and fall have the greater weekly_sales. may it is because of christmas and blackfriday

In [ ]:
# winter_df = train[train.season == "Winter"][['weekly_sales', 'temperature', 'season']]
# winter_df.head()

In [ ]:
# sns.jointplot(x= 'weekly_sales', y= 'temperature', data = winter_df, hue= "season")

In [ ]:
train.season.unique()

In [ ]:
color_list = ['blue','green', 'red', 'orange']

In [ ]:
for season, color in zip(train.season.unique(), color_list):
    print(season, color)

In [ ]:

for season, c in zip(train.season.unique(), color_list):
    print(season)
    sns.jointplot(x= 'weekly_sales', y= 'temperature', data = train[train.season==season], color= c)
    plt.show()

### -  weekly_sales vs holiday_name

In [ ]:
plt.figure(figsize=(12,10))
sns.boxplot(x= train['holiday_name'], y =train['weekly_sales'] )
plt.show()

**Takeaways**
- the 2 holidays with the greatest weekle_sales are pre_christmas and thanksgiving
-we can see christmas does not have large amount of weekly_sales it is because the people buy before christmas , it is not like blackfriday that the people actually buy in this specific week

### - weekly_sales vs season

In [ ]:
plt.figure(figsize=(12,10))
sns.boxplot(x= train['season'], y =train['weekly_sales'] )

**takeaways**
- we see some outliers but basically there are the weekly sales for thanksgiving and christmas (pre)
- we can say summer has the lowest range of weekly sales amout

In [ ]:
for season, c in zip(train.season.unique(), color_list):
    print(season)
    sns.boxplot(x= train[train.season== season]['weekly_sales'], color = c )
    plt.show()

**takeaways**
- it looks like summer has the lowest median 

<hr style="border:2px solid black"> </hr>

In [ ]:
X_train_scaled.head(1).T

# model

In [9]:
#imports
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.linear_model import LinearRegression, LassoLars, TweedieRegressor 
from sklearn.feature_selection import RFE
from sklearn.preprocessing import MinMaxScaler


from sklearn.preprocessing import PolynomialFeatures

In [10]:

# #conver y_train y _ validate to df
y_train_df = pd.DataFrame( {'actual': y_train})
y_test_df = pd.DataFrame( {'actual': y_test})

In [11]:
def select_kbest  (X_df, y_df, n_features):
    '''
    Takes in the predictors, the target, and the number of features to select (k) ,
    and returns the names of the top k selected features based on the SelectKBest class
    
    X_df : the predictors
    y_df : the target
    n_features : the number of features to select (k)
    Example
    select_kbest(X_train_scaled, y_train, 2)
    '''
    
    f_selector = SelectKBest(score_func=f_regression, k= n_features)
    f_selector.fit(X_df, y_df)
    mask = f_selector.get_support()
    X_df.columns[mask]
    top = list(X_df.columns[mask])
    print(f'The top {n_features} selected feautures based on the SelectKBest class are: {top}' )
    return top

In [12]:
def select_rfe (X_df, y_df, n_features, method):
    '''
    Takes in the predictors, the target, and the number of features to select (k) ,
    and returns the names of the top k selected features based on the Recursive Feature Elimination (RFE)
    
    X_df : the predictors
    y_df : the target
    n_features : the number of features to select (k)
    method : LinearRegression, LassoLars, TweedieRegressor
    Example
    select_rfe(X_train_scaled, y_train, 2, LinearRegression())
    '''
    lm = method
    rfe = RFE(estimator=lm, n_features_to_select= n_features)
    rfe.fit(X_df, y_df)
    top_rfe = list(X_df.columns[rfe.support_])
    print(f'The top {n_features} selected feautures based on the the RFE class class are: {top_rfe}' )
    print(pd.Series(dict(zip(X_df.columns, rfe.ranking_))).sort_values())
    return top_rfe


In [13]:
def regression_errors(df, y, yhat):
    '''
    Takes in a dataframe , y = column with actual_values and yhat= name of the columns with predicted_values
    and calculate:
    sum of squared errors (SSE)
    explained sum of squares (ESS)
    total sum of squares (TSS)
    mean squared error (MSE)
    root mean squared error (RMSE)
    Returns a dictionary with all these values.
    Example:
    plot_residuals(df, 'tip', 'yhat')
    '''
    #import
    from sklearn.metrics import  mean_squared_error
    from math import sqrt
    
    
    #calculate SSE using sklearn
    SSE = mean_squared_error(df[y], df[yhat])*len(df)
    #explained sum of squares (ESS)
    ESS = ((df[yhat] - df[y].mean())**2).sum()
    #total sum of squares (TSS)
    TSS = ((df[y] - df[y].mean())**2).sum()
    #mean squared error (MSE)
    MSE = mean_squared_error(df[y], df[yhat])
    #root mean squared error (RMSE)
    RMSE = sqrt(MSE)
    
    #create a dictionary
    m= {
        'sse': SSE,
        'ess': ESS,
        'rmse': RMSE,
        'tss': TSS,
        'mse': MSE,
        'r2': ESS/TSS,
    }

    return m

In [14]:
def create_model (X_df_scaled, y_df, actual, method, name):
    '''
    takes in features scaled df, target df, name of actual target, 
    type of method and the name of the selected method and 
    returns a dictionary that contains calculated regression errors.
    
    X_df_scaled : df that contains scaled featues
    y_df: target df
    actual: name of the column where is actual value of the target
    mehod: type of method to create the model object
    name: enter the new name for your model
    
    Example:
    create_model(X_train_scaled[top_sb], y_train, 'actual', LinearRegression(normalize=True), 'modelOLS' )
    '''
    # fit the thing
    method.fit(X_df_scaled, y_df[actual])

    # predict train
    y_df[name] = method.predict(X_df_scaled)

    #calculate regression errors using a created function
    train_eval = regression_errors(y_df, actual, name)

    return train_eval

In [15]:
def report(metric_df):
    
    from IPython.display import display, HTML
    rmse_base = metric_df.iloc[0,2]
    print(f'These are the models that perform better than our baseline rmse: {rmse_base}')
    dfs =metric_df[['model', 'rmse_validate']][metric_df['rmse_validate'] < rmse_base]
    display(HTML(dfs.to_html()))
    
    
    min_val = metric_df['rmse_validate'].idxmin()
    metric_df.iloc[min_val][0]
    rsme_bet = round(metric_df['rmse_validate'].iloc[min_val], 2)
    print('-----------------------------------------------------------------------------------------------')
    print(f'   ********** The model with the less  rmse_validate  is {metric_df.iloc[min_val][0] }  rmse:{rsme_bet} **********             ')
    print('-----------------------------------------------------------------------------------------------')
    print(' ')
    min_val = metric_df['r^2_validate'].idxmax()
    metric_df.iloc[min_val][0]
    print(f'The model with r^2 validate closer to 1 is ', metric_df.iloc[min_val][0])
    
    display(HTML(metric_df.to_html()))
    return

In [16]:
def baseline_errors(df, y, option ):
    '''
    Takes in a dataframe , y = column with actual_values 
    and calculate:
    sum of squared errors (SSE)
    explained sum of squares (ESS)
    total sum of squares (TSS)
    mean squared error (MSE)
    root mean squared error (RMSE)
    Returns a dictionary with all these values
    Example:
    baseline_mean_errors(y_train, 'actual')
    '''
    #import
    from sklearn.metrics import  mean_squared_error
    from math import sqrt

    #baseline
    if option == 'mean':
        df['yhat_baseline_mean'] = df[y].mean()
        col = 'yhat_baseline_mean'
    elif option == 'median':
        df['yhat_baseline_median'] = df[y].median()
        col = 'yhat_baseline_median'
    else:
        return print("please select the correct option: 'mean' or 'median' ")
        
    

    #calculate SSE using sklearn
    SSE_baseline = mean_squared_error(df[y], df[col])*len(df)
    #explained sum of squares (ESS)
    ESS_b = ((df[col] - df[y].mean())**2).sum()
    #total sum of squares (TSS)
    TSS_b = ((df[y] - df[y].mean())**2).sum()
    #mean squared error (MSE)
    MSE_baseline = mean_squared_error(df[y], df[col])
    #root mean squared error (RMSE)
    RMSE_baseline = sqrt(MSE_baseline)
    
    #create dicc
    b ={
        'sse': SSE_baseline,
        'mse': MSE_baseline,
        'rmse': RMSE_baseline,
         'tss': TSS_b,
        'ess' : ESS_b,
        'mse': MSE_baseline,
        'r2': ESS_b/TSS_b,       
    }

    return b

**- Baseline**

In [ ]:
# #create baseline using mean (I'm using my function to calculate rmse)
# tra = baseline_errors(y_train_df, 'actual', 'mean')
# tra['rmse']

In [ ]:
# #create a df to store metrics
# metric_df = pd.DataFrame(data = [{
#     'model': 'mean_baseline',
#     'rmse_train' : round(tra['rmse'], 2),
#     'r^2' : 0}])

In [20]:

#baseline version 2 using last years sales
y_train_df['last_year_baseline'] = train['next_week_1_year_ago']

In [21]:
import math
from sklearn.metrics import mean_squared_error, explained_variance_score
#calculate RMSE for baseline model
rmse_baseline2_train= math.sqrt(mean_squared_error(y_train_df.actual, y_train_df.last_year_baseline))

In [22]:
rmse_baseline2_train

91145.28223498359

In [30]:
#add the mretic to our df
metric_df = pd.DataFrame(data = [{
    'model': 'baseline(using last year sales)',
    'rmse_train':rmse_baseline2_train,    
    'r^2' : 0}])

In [31]:
metric_df

,model,rmse_train,r^2
0,baseline(using last year sales),91145.282235,0


### - select k best

### - cross validation

In [32]:
from sklearn.model_selection import cross_val_score, GridSearchCV, train_test_split

In [33]:
#create the model
clf = LinearRegression(normalize=True)
#cv = number of folds
cross_val_score(clf, X_train_scaled, y_train, cv=3)

array([0.98419954, 0.98313829, 0.98420746])

In [34]:
import sklearn
sklearn.metrics.SCORERS.keys()

dict_keys(['explained_variance', 'r2', 'max_error', 'neg_median_absolute_error', 'neg_mean_absolute_error', 'neg_mean_squared_error', 'neg_mean_squared_log_error', 'neg_root_mean_squared_error', 'neg_mean_poisson_deviance', 'neg_mean_gamma_deviance', 'accuracy', 'roc_auc', 'roc_auc_ovr', 'roc_auc_ovo', 'roc_auc_ovr_weighted', 'roc_auc_ovo_weighted', 'balanced_accuracy', 'average_precision', 'neg_log_loss', 'neg_brier_score', 'adjusted_rand_score', 'homogeneity_score', 'completeness_score', 'v_measure_score', 'mutual_info_score', 'adjusted_mutual_info_score', 'normalized_mutual_info_score', 'fowlkes_mallows_score', 'precision', 'precision_macro', 'precision_micro', 'precision_samples', 'precision_weighted', 'recall', 'recall_macro', 'recall_micro', 'recall_samples', 'recall_weighted', 'f1', 'f1_macro', 'f1_micro', 'f1_samples', 'f1_weighted', 'jaccard', 'jaccard_macro', 'jaccard_micro', 'jaccard_samples', 'jaccard_weighted'])

In [35]:
#create the model
clf = LinearRegression(normalize=True)
#cv = number of folds
cross_val_score(clf, X_train_scaled, y_train, cv=5, scoring = 'r2')

array([0.98599044, 0.98226797, 0.98253562, 0.98596136, 0.98314991])

In [36]:
#create the model
clf = LinearRegression(normalize=True)
#cv = number of folds
cross_val_score(clf, X_train_scaled, y_train, cv=3, scoring = 'neg_root_mean_squared_error')

array([-69529.70428164, -71656.45914894, -71583.06247297])

In [37]:
#create the model
clf = LinearRegression(normalize=False)
#cv = number of folds
cross_val_score(clf, X_train_scaled, y_train, cv=3)

array([0.98419954, 0.98313829, 0.98420746])

## GridSearchCV

In [38]:
from sklearn.model_selection import GridSearchCV

#specify the parameters we wish to use as a dictionary, then use that dictionary when we create the class.
params = {'normalize': [ True, False],
          'fit_intercept': [True, False]}

ols = LinearRegression()

grid = GridSearchCV(ols, params, cv=5, scoring='neg_root_mean_squared_error')

grid.fit(X_train_scaled, y_train)

GridSearchCV(cv=5, estimator=LinearRegression(),
             param_grid={'fit_intercept': [True, False],
                         'normalize': [True, False]},
             scoring='neg_root_mean_squared_error')

In [39]:
#see the cross validation results in the cv_results_ property of the object we created.
results = grid.cv_results_
results.keys()

dict_keys(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time', 'param_fit_intercept', 'param_normalize', 'params', 'split0_test_score', 'split1_test_score', 'split2_test_score', 'split3_test_score', 'split4_test_score', 'mean_test_score', 'std_test_score', 'rank_test_score'])

In [40]:
# I will use mean_test_score
test_scores = results['mean_test_score']
test_scores

array([-70476.1904101 , -70476.1904101 , -72272.95355196, -72272.95355196])

In [41]:
params = results['params']
params

[{'fit_intercept': True, 'normalize': True},
 {'fit_intercept': True, 'normalize': False},
 {'fit_intercept': False, 'normalize': True},
 {'fit_intercept': False, 'normalize': False}]

In [42]:
#We can combine these features together into a data frame to see how our different models perform:
for p, s in zip(params, test_scores):
    p['score'] = s

pd.DataFrame(params).sort_values(by='score')

,fit_intercept,normalize,score
2,False,True,-72272.953552
3,False,False,-72272.953552
1,True,False,-70476.190410
0,True,True,-70476.190410


In [43]:
def gridsearch ( X_df , y_df , model, params, score):
    '''
    '''
    grid = GridSearchCV(model, params, cv=5, scoring= score )
    grid.fit(X_df, y_df)
    #see the cross validation results in the cv_results_ property of the object we created.
    results = grid.cv_results_
    # I will use mean_test_score
    test_scores = results['mean_test_score']
    #GETTING THE PARAMETERS
    params = results['params']
    #We can combine these features together into a data frame to see how our different models perform:
    for p, s in zip(params, test_scores):
        p['score'] = s

    return pd.DataFrame(params).sort_values(by='score')

##  LinearRegression (OLS)

### - Gridsearch

In [54]:
#specify the parameters we wish to use as a dictionary, then use that dictionary when we create the class.
params = {'normalize': [ True, False],
          'fit_intercept': [True, False]}

In [57]:
#use a gridsearch function using 
gridsearch (X_train_scaled, y_train, LinearRegression() , params, 'neg_root_mean_squared_error')

,fit_intercept,normalize,score
2,False,True,-72272.953552
3,False,False,-72272.953552
1,True,False,-70476.190410
0,True,True,-70476.190410


### OLS uising  select K best (6)

In [68]:
#using my function for SelectkBest
top_sb =select_kbest(X_train_scaled, y_train, 10)

The top 10 selected feautures based on the SelectKBest class are: ['this_week_sales', 'CPI', 'this_week_unemployment', 'store_size', 'next_week_1_year_ago', 'pre_christmas', 'cpi_4wk_rolling', 'unemp_4wk_rolling', 'cpi_quarterly_rolling', 'unemp_quarterly_rolling']


In [69]:
#create the model
ols_sb = create_model(X_train_scaled[top_sb], y_train_df, 'actual',\
                       LinearRegression(normalize=True,\
                        fit_intercept=True ), 'modelOLS' )
ols_sb['rmse']


73278.63464117885

#### cross validation

In [70]:
#create the model
clf = LinearRegression(normalize=True, fit_intercept=True )
#cv = number of folds
cross_val_score(clf, X_train_scaled[top_sb], y_train, cv=3, scoring = 'neg_root_mean_squared_error')

array([-73828.51208786, -73408.20028783, -74244.87809008])

### OLS uising  RFE (8)

In [73]:
#uise the fucntion to get RFE
top_rfe = select_rfe(X_train_scaled, y_train, 10,LinearRegression(normalize=True, fit_intercept=True ))

The top 10 selected feautures based on the the RFE class class are: ['this_week_sales', 'CPI', 'this_week_unemployment', 'next_week_1_year_ago', 'christmas', 'thanksgiving', 'cpi_4wk_rolling', 'avgMoM_perc_unemp', 'cpi_quarterly_rolling', 'unemp_quarterly_rolling']
this_week_sales             1
unemp_quarterly_rolling     1
cpi_quarterly_rolling       1
avgMoM_perc_unemp           1
CPI                         1
this_week_unemployment      1
cpi_4wk_rolling             1
next_week_1_year_ago        1
thanksgiving                1
christmas                   1
avgQoQ_perc_unemp           2
store_size                  3
fuel_quarterly_rolling      4
avgMoM_perc_fuel            5
fuel_4wk_rolling            6
avgQoQ_perc_fuel            7
this_week_holiday_flag      8
fuel_price                  9
unemp_4wk_rolling          10
avgMoM_perc_cpi            11
labor_day                  12
super_bowl                 13
temperature                14
avgQoQ_perc_cpi            15
pre_christmas 

In [74]:
#create the model
ols_rfe = create_model(X_train_scaled[top_rfe], y_train_df, 'actual', LinearRegression(normalize=True, fit_intercept=True ), 'modelOLS' )
ols_rfe['rmse']

71859.75745004068

### - cross validation

In [75]:
#create the model
clf = LinearRegression(normalize=True, fit_intercept=True )
#cv = number of folds
cross_val_score(clf, X_train_scaled[top_rfe], y_train, cv=3, scoring = 'neg_root_mean_squared_error')

array([-72127.07288044, -73241.00581704, -72267.9117694 ])

Note: we are going to use ols_rfe  model

In [76]:
#add the mretic to our df
metric_df = metric_df.append(
    {
    'model': 'ols_rfe',
    'rmse_train': ols_rfe['rmse'],    
    'r^2' : ols_rfe['r2']}, ignore_index=True)

In [77]:
metric_df

,model,rmse_train,r^2
0,baseline(using last year sales),91145.282235,0.000000
1,ols_rfe,71859.757450,0.983443


## LassoLars

### Gridsearch

In [78]:
#specify the parameters we wish to use as a dictionary, then use that dictionary when we create the class.
params = {
          'normalize': [True, False],
          'fit_intercept':[True, False],
           'alpha': [1.0, 0]
         }

In [79]:
#use the function to get the combinations of parameters
gridsearch (X_train_scaled, y_train, LassoLars() , params, 'neg_root_mean_squared_error')

,alpha,fit_intercept,normalize,score
2,1.0,False,True,-72298.372878
3,1.0,False,False,-72298.372878
6,0.0,False,True,-72272.953552
7,0.0,False,False,-72272.953552
5,0.0,True,False,-70476.190410
4,0.0,True,True,-70476.190410
0,1.0,True,True,-70458.146015
1,1.0,True,False,-70440.674321


### RFE 

In [83]:
#get the 9 features
top_rfe = select_rfe(X_train_scaled, y_train, 6, LassoLars(alpha = 1, normalize= False, fit_intercept= True) )

The top 6 selected feautures based on the the RFE class class are: ['this_week_sales', 'this_week_unemployment', 'next_week_1_year_ago', 'christmas', 'thanksgiving', 'unemp_quarterly_rolling']
this_week_sales             1
unemp_quarterly_rolling     1
thanksgiving                1
christmas                   1
this_week_unemployment      1
next_week_1_year_ago        1
avgMoM_perc_unemp           2
avgQoQ_perc_unemp           3
store_size                  4
avgMoM_perc_fuel            5
fuel_quarterly_rolling      6
fuel_4wk_rolling            7
avgQoQ_perc_fuel            8
this_week_holiday_flag      9
avgMoM_perc_cpi            10
fuel_price                 11
next_week_holiday_flag     12
temperature                13
CPI                        14
avgQoQ_perc_cpi            15
pre_christmas              16
super_bowl                 17
cpi_4wk_rolling            18
unemp_4wk_rolling          19
labor_day                  20
cpi_quarterly_rolling      21
dtype: int64


In [84]:
#create the model
lasso_rfe = create_model(X_train_scaled[top_rfe], y_train_df, 'actual', LassoLars(alpha = 1, normalize= False, fit_intercept= True), 'modelLasso' )
lasso_rfe['rmse']

72080.26921283794

### -cross validation 

In [85]:
#create the model
clf = LassoLars(alpha = 1, normalize= False, fit_intercept= True)
#cv = number of folds
cross = cross_val_score(clf, X_train_scaled[top_rfe], y_train, cv=3, scoring = 'neg_root_mean_squared_error')
cross

array([-72369.74422049, -73054.01907945, -72590.01601532])

### select K best

In [86]:
#using my function for SelectkBest
top_sb =select_kbest(X_train_scaled, y_train, 10)

The top 10 selected feautures based on the SelectKBest class are: ['this_week_sales', 'CPI', 'this_week_unemployment', 'store_size', 'next_week_1_year_ago', 'pre_christmas', 'cpi_4wk_rolling', 'unemp_4wk_rolling', 'cpi_quarterly_rolling', 'unemp_quarterly_rolling']


In [87]:
lasso_skb = create_model(X_train_scaled[top_sb], y_train_df, 'actual', LassoLars(alpha = 1, normalize= False, fit_intercept= True), 'modelLasso' )
lasso_skb['rmse']

73281.87552494287

In [88]:
#create the model
clf = LassoLars(alpha = 1, normalize= False, fit_intercept= True)
#cv = number of folds
cross= cross_val_score(clf, X_train_scaled[top_sb], y_train, cv=3, scoring = 'neg_root_mean_squared_error')
cross

array([-73875.3247233 , -73345.36555372, -74270.0989156 ])

In [89]:
#add the best model and its metrics
metric_df = metric_df.append(
    {
    'model': 'lasso_rfe',
    'rmse_train': lasso_rfe['rmse'],    
    'r^2' : lasso_rfe['r2']}, ignore_index=True)

In [90]:
metric_df

,model,rmse_train,r^2
0,baseline(using last year sales),91145.282235,0.000000
1,ols_rfe,71859.757450,0.983443
2,lasso_rfe,72080.269213,0.983309



## TweedieRegressor (GLM)

### - GridSearch



In [91]:
#specify the parameters we wish to use as a dictionary, then use that dictionary when we create the class.
params = {
          'power': [0.0, 1],
           'fit_intercept' : [True , False],
          'warm_start': [True, False], 
           'alpha': [1.0, 0.0]
         }

In [92]:
gridsearch (X_train_scaled, y_train,TweedieRegressor() , params, 'neg_root_mean_squared_error')

,alpha,fit_intercept,power,warm_start,score
6,1.0,False,1.0,True,-1.109649e+06
7,1.0,False,1.0,False,-1.109649e+06
14,0.0,False,1.0,True,-1.106117e+06
15,0.0,False,1.0,False,-1.106117e+06
4,1.0,False,0.0,True,-5.226843e+05
5,1.0,False,0.0,False,-5.226843e+05
0,1.0,True,0.0,True,-4.957444e+05
1,1.0,True,0.0,False,-4.957444e+05
2,1.0,True,1.0,True,-1.442832e+05
3,1.0,True,1.0,False,-1.442832e+05


### create model using RFE

In [99]:
#use function to get the top 6 RFE
top_rfe = select_rfe(X_train_scaled, y_train, 6, TweedieRegressor(alpha =0 , fit_intercept= True, power=0 ,\
                                                                  warm_start= False) )

The top 6 selected feautures based on the the RFE class class are: ['this_week_sales', 'store_size', 'next_week_1_year_ago', 'christmas', 'thanksgiving', 'avgQoQ_perc_unemp']
this_week_sales             1
thanksgiving                1
christmas                   1
next_week_1_year_ago        1
store_size                  1
avgQoQ_perc_unemp           1
avgMoM_perc_fuel            2
fuel_quarterly_rolling      3
fuel_4wk_rolling            4
avgQoQ_perc_fuel            5
unemp_4wk_rolling           6
this_week_unemployment      7
this_week_holiday_flag      8
avgMoM_perc_cpi             9
unemp_quarterly_rolling    10
avgMoM_perc_unemp          11
labor_day                  12
super_bowl                 13
temperature                14
fuel_price                 15
avgQoQ_perc_cpi            16
cpi_quarterly_rolling      17
next_week_holiday_flag     18
CPI                        19
pre_christmas              20
cpi_4wk_rolling            21
dtype: int64


In [100]:
#create the model
gml_rfe = create_model(X_train_scaled[top_rfe], y_train_df, 'actual',TweedieRegressor(alpha =0 , fit_intercept= True, power=0 ,\
                                                                  warm_start= False), 'modelgml' )
gml_rfe['rmse']

71522.6640373484

### - cross validation

In [101]:
#create the model
clf = TweedieRegressor(alpha =0 , fit_intercept= True, power=0 ,warm_start= False)
#cv = number of folds
cross = cross_val_score(clf, X_train_scaled[top_rfe], y_train, scoring = 'neg_root_mean_squared_error', cv=3)
cross

array([-71446.65517559, -72245.35839113, -72581.66333012])

### create model using select kbest

In [105]:
#using my function for SelectkBest
top_sb =select_kbest(X_train_scaled, y_train, 6)

The top 6 selected feautures based on the SelectKBest class are: ['this_week_sales', 'this_week_unemployment', 'store_size', 'next_week_1_year_ago', 'unemp_4wk_rolling', 'unemp_quarterly_rolling']


In [106]:
gml_skb = create_model(X_train_scaled[top_sb], y_train_df, 'actual',TweedieRegressor(alpha =0 , fit_intercept= True, power=0 ,\
                                                                  warm_start= False), 'modelgml' )
gml_skb['rmse']

73447.13066759182

In [107]:
#create the model
clf = TweedieRegressor(alpha =0 , fit_intercept= True, power=0 ,warm_start= False)
#cv = number of folds
cross = cross_val_score(clf, X_train_scaled[top_sb], y_train, scoring = 'neg_root_mean_squared_error',cv=3)
cross

array([-74054.54562594, -73200.00828644, -74368.20223621])

In [108]:
#add the best model
metric_df = metric_df.append(
    {
    'model': 'gml_rfe',
    'rmse_train': gml_rfe['rmse'],    
    'r^2' : gml_rfe['r2']}, ignore_index=True)

In [109]:
metric_df

,model,rmse_train,r^2
0,baseline(using last year sales),91145.282235,0.000000
1,ols_rfe,71859.757450,0.983443
2,lasso_rfe,72080.269213,0.983309
3,gml_rfe,71522.664037,0.983598


## plynomial Regression

###  select k best 

In [124]:
#using my function for SelectkBest
top_sb =select_kbest(X_train_scaled, y_train, 6)

The top 6 selected feautures based on the SelectKBest class are: ['this_week_sales', 'this_week_unemployment', 'store_size', 'next_week_1_year_ago', 'unemp_4wk_rolling', 'unemp_quarterly_rolling']


In [125]:
# make the polynomial features to get a new set of features
pf = PolynomialFeatures(degree = 3) 

# fit and transform X_train_scaled
X_train_degree3 = pf.fit_transform(X_train_scaled[top_sb])

# transform X_validate_scaled & X_test_scaled
X_test_degree3 = pf.transform(X_test_scaled[top_sb])

### GridSearch

In [126]:
#specify the parameters we wish to use as a dictionary, then use that dictionary when we create the class.
params = {
          'normalize': [True, False],
          'fit_intercept':[True, False],
         }

In [127]:
gridsearch (X_train_degree3, y_train, LinearRegression() , params, 'neg_root_mean_squared_error')

,fit_intercept,normalize,score
0,True,True,-68367.964689
1,True,False,-68367.964689
2,False,True,-68367.964689
3,False,False,-68367.964689


In [128]:
gridsearch (X_train_degree3, y_train, LinearRegression() , params, 'r2')

,fit_intercept,normalize,score
0,True,True,0.984925
1,True,False,0.984925
2,False,True,0.984925
3,False,False,0.984925


In [129]:
pol_skb = create_model(X_train_degree3, y_train_df, 'actual',LinearRegression( normalize=False, fit_intercept = False ), 'modelpol' )
pol_skb['rmse']

64044.02037218975

In [130]:
#create the model
clf = LinearRegression( normalize=False, fit_intercept = True )
#cv = number of folds
cross = cross_val_score(clf, X_train_degree3, y_train, cv=3, scoring = 'neg_root_mean_squared_error')
cross

array([-66722.79374691, -70656.60660325, -72587.28808291])

In [131]:
metric_df = metric_df.append(
    {
    'model': 'pol3_skb',
    'rmse_train': pol_skb['rmse'],    
    'r^2' : pol_skb['r2']}, ignore_index=True)

### RFE

In [133]:
 top_rfe = select_rfe(X_train_scaled, y_train, 7, LinearRegression(normalize=False, fit_intercept = False) )

The top 7 selected feautures based on the the RFE class class are: ['this_week_sales', 'CPI', 'next_week_1_year_ago', 'cpi_4wk_rolling', 'unemp_4wk_rolling', 'cpi_quarterly_rolling', 'unemp_quarterly_rolling']
this_week_sales             1
unemp_quarterly_rolling     1
cpi_quarterly_rolling       1
CPI                         1
next_week_1_year_ago        1
unemp_4wk_rolling           1
cpi_4wk_rolling             1
this_week_unemployment      2
avgMoM_perc_unemp           3
avgMoM_perc_cpi             4
christmas                   5
thanksgiving                6
fuel_price                  7
fuel_4wk_rolling            8
store_size                  9
avgQoQ_perc_unemp          10
this_week_holiday_flag     11
avgQoQ_perc_fuel           12
avgQoQ_perc_cpi            13
avgMoM_perc_fuel           14
temperature                15
super_bowl                 16
pre_christmas              17
fuel_quarterly_rolling     18
labor_day                  19
next_week_holiday_flag     20
dtype: int

In [134]:
# make the polynomial features to get a new set of features
pf = PolynomialFeatures(degree = 3) 

# fit and transform X_train_scaled
X_train_degree3 = pf.fit_transform(X_train_scaled[top_rfe])

# transform X_validate_scaled & X_test_scaled
X_test_degree3 = pf.transform(X_test_scaled[top_rfe])

In [135]:
gridsearch (X_train_degree3, y_train, LinearRegression() , params, 'neg_root_mean_squared_error')

,fit_intercept,normalize,score
0,True,True,-65466.028958
2,False,True,-64896.793744
3,False,False,-64896.793744
1,True,False,-64896.793729


In [136]:
#create the model
pol_rfe = create_model(X_train_degree3, y_train_df, 'actual',LinearRegression( normalize=False, fit_intercept = True ), 'modelpol' )
pol_rfe['rmse']

60488.51113347658

In [137]:
#create the model
clf = LinearRegression( normalize=True, fit_intercept = True )
#cv = number of folds
cross = cross_val_score(clf, X_train_degree3, y_train, cv=3, scoring = 'neg_root_mean_squared_error')
cross

array([-65122.10841691, -67271.31287954, -66561.22897066])

In [138]:
metric_df = metric_df.append(
    {
    'model': 'pol3_rfe',
    'rmse_train': pol_rfe['rmse'],    
    'r^2' : pol_rfe['r2']}, ignore_index=True)

### Degree 2  with all features

In [149]:
# make the polynomial features to get a new set of features using the entire df
pf = PolynomialFeatures(degree = 1) 

# fit and transform X_train_scaled
X_train_degree2 = pf.fit_transform(X_train_scaled)

# transform X_validate_scaled & X_test_scaled
X_test_degree2 = pf.transform(X_test_scaled)

In [150]:
gridsearch (X_train_degree2, y_train, LinearRegression() , params, 'neg_root_mean_squared_error')

,fit_intercept,normalize,score
2,False,True,-70476.19041
3,False,False,-70476.19041
1,True,False,-70476.19041
0,True,True,-70476.19041


In [151]:
pol_dg2 = create_model(X_train_degree2, y_train_df, 'actual',LinearRegression( normalize=True, fit_intercept = True ), 'modelpol' )
pol_dg2['rmse']

69757.43755006164

In [153]:
#create the model
clf = LinearRegression( normalize=True, fit_intercept = True )
#cv = number of folds
cross = cross_val_score(clf, X_train_degree2, y_train, cv=3, scoring = 'neg_root_mean_squared_error')
cross

array([-69529.70428164, -71656.45914894, -71583.06247297])

In [154]:
metric_df = metric_df.append(
    {
    'model': 'pol_dg2',
    'rmse_train': pol_dg2['rmse'],    
    'r^2' : pol_dg2['r2']}, ignore_index=True)

In [155]:
metric_df

,model,rmse_train,r^2
0,baseline(using last year sales),91145.282235,0.000000
1,ols_rfe,71859.757450,0.983443
2,lasso_rfe,72080.269213,0.983309
3,gml_rfe,71522.664037,0.983598
4,pol3_skb,64044.020372,0.986849
5,pol3_rfe,60488.511133,0.988269
6,pol_dg2,69757.437550,0.984398


#  Test

In [156]:
#use my function to create and calculate the metrics
pol_reg_test =  create_model(X_test_degree3, 
                              y_test_df, 'actual',LinearRegression( normalize=False, fit_intercept = True ), 'test_polreg' )

In [157]:
pol_reg_test['rmse']

62577.99078326525